In [157]:
import pandas as pd
import os
import re

In [341]:
#This function takes is a string (name of item,npc,location,weapon,etc) and returns the dataframe where it is located
#Example: Passing in "Giant Bat" will return the creatures.csv dataframe, passing in a weapon name will return the weapons.csv dataframe and so on


def find_dataset(name):
    arr = os.listdir('data/')
    
    for i in range(len(arr)):
        
        if ".csv" in arr[i]:
            df = pd.read_csv("data/" +arr[i])
            
            for value in df["name"]:
                
                if name.lower() in value.lower():
                    return df
            else:
                return "No matching "
                
#This function uses find_dataset to grab the dataframe of the passed in string, then returns the row where the string matches
def search(search_string):  
    df = find_dataset(search_string)
    return df[df["name"]==search_string]

#This function takes in a type of data and returns the entire frame
#Example: Passing "armors" will return the entire armors.csv dataframe
def sets(type_str):
        
    df = pd.read_csv("data/" + type_str+ ".csv")
    
    return df
    

In [353]:
def parse_atk_def(df,atk_or_def):

    print(df.iloc[6])
    #takes in row passed in from weapons_df
    #loops through df["attack"], matches and saves type of dmg in list
    for i in df.iloc[atk_or_def]:
        types = re.findall(r"(?:[A-Z][a-z]+)", i)
    
    #loops through df["attack"], matches and saves amount of dmg in list
    for k in df.iloc[atk_or_def]:
        dmg = re.findall(r"(?:\d+)", k)
    
    for atk_type in range(len(types)):            
        
        #adds "_atk" or "_def" to all values in list
        if atk_or_def == 4:
            lists = [[s + "_atk" for s in types], dmg]
        elif atk_or_def == 5:
            lists = [[s + "_def" for s in types], dmg]
            
        #concats the types and dmg dataframes, transposes to flat format
        df = df.reset_index(drop=True)
        df = pd.concat([pd.Series(x) for x in lists], axis=1)
        df = df.T

        df = df.rename(columns=df.iloc[0])[1:]
        return df
    
def create_atk_def(df,atk_or_def):
    #create empty df to append to
    new_df = pd.DataFrame()
    #iterating through rows in the weapons_df
    for row in df.iterrows():
        
        #apply parse_atk function to row, append returning df of attack type + dmg amount to empty df
        current_row = row[1].to_frame()
        #print(current_row)
        new_df = new_df.append(parse_atk_def(current_row,atk_or_def))
    #print(atk(row[1].to_frame()))
    #drops empty nan
    try:
        new_df = new_df.drop(columns=[float("nan")])
        return new_df.reset_index(drop=True)
    except:
        return new_df.reset_index(drop=True)

    

In [ ]:
def parse_scaling(df):
    #takes in row passed in from weapons_df
    #loops through df["scalesWith"], matches and saves scaling in list
    #print(df)
    scales_with = []
    for i in df["scalesWith"]:
        #print(i)
        scales_with.append(re.findall(r"(?:[A-Z][a-z][a-z])", i))
    #print(scales_with)
    #loops through df["scalesWith"], matches and saves scalings in list
    clean_scalings = []
    for k in df["scalesWith"]:
        clean_scalings.append(re.findall(r"(?:[A-E])\'", k))

        #for i in scaling:
        #    clean_scalings.append(i[0])
    
    
    #print(clean_scalings)


    #lists = [scales_with,clean_scalings]  
    #df = pd.concat([pd.Series(x) for x in lists], axis=1, ignore_index=True)
    #df = df.T
    #print(df)
    
    #df.columns = df.iloc[0]
    #df = df[1:]
    #df["scalesWith"] = scales_with
    #df["scaling_rank"] = clean_scalings
    #df = df.rename(columns=df.iloc[0])[1:]
    #print(df)
    return (scales_with,clean_scalings)

def add_scaling(df):
    scale = parse_scaling(df)

    scale_with = scale[0]
    scaling_rate = scale[1]
    #df["scalesWith"] = "0"
    df["scalesWith"] = scale_with
    df["scalingRate"] = scaling_rate
    return df

In [354]:
def merge_dfs(df):
    return pd.concat([df, create_atk_def(df,4), create_atk_def(df,5)], axis=1)

weapons_df = sets("weapons")
merge_dfs(weapons_df)

0    [{'name': 'Str', 'scaling': 'D'}, {'name': 'De...
Name: scalesWith, dtype: object
1    [{'name': 'Str', 'scaling': 'D'}, {'name': 'De...
Name: scalesWith, dtype: object
2    [{'name': 'Str', 'scaling': 'D'}, {'name': 'De...
Name: scalesWith, dtype: object
3    [{'name': 'Str', 'scaling': 'C'}, {'name': 'De...
Name: scalesWith, dtype: object
4    [{'name': 'Str', 'scaling': 'D'}, {'name': 'De...
Name: scalesWith, dtype: object
5    [{'name': 'Arc', 'scaling': 'A'}]
Name: scalesWith, dtype: object
6    [{'name': 'Str', 'scaling': 'E'}, {'name': 'De...
Name: scalesWith, dtype: object
7    [{'name': 'Str', 'scaling': 'D'}, {'name': 'De...
Name: scalesWith, dtype: object
8    [{'name': 'Str', 'scaling': '?'}, {'name': 'De...
Name: scalesWith, dtype: object
9    [{'name': 'Str', 'scaling': 'D'}, {'name': 'De...
Name: scalesWith, dtype: object
10    [{'name': 'Str', 'scaling': 'D'}, {'name': 'De...
Name: scalesWith, dtype: object
11    [{'name': 'Str', 'scaling': 'C'}]
Name: scalesWith, 

,id,name,image,description,attack,defence,scalesWith,requiredAttributes,category,weight,...,Sor_atk,Inc_atk,Sorc_atk,Light_atk,Phy_def,Mag_def,Fire_def,Ligt_def,Holy_def,Boost_def
0,17f69c35d2cl0i1oh7zuqfb3mdvsj,Hand Axe,https://eldenring.fanapis.com/images/weapons/1...,"Commonly known as a hatchet, this smaller vari...","[{'name': 'Phy', 'amount': 113}, {'name': 'Mag...","[{'name': 'Phy', 'amount': 42}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'D'}, {'name': 'De...","[{'name': 'Str', 'amount': 9}, {'name': 'Dex',...",Axe,3.5,...,NaN,NaN,NaN,NaN,42,28,28,28,28,28
1,17f69667224l0i1ohar280aeoxkl6z,Jawbone Axe,https://eldenring.fanapis.com/images/weapons/1...,Axe made from a herbivore's skull. Weapon of t...,"[{'name': 'Phy', 'amount': 124}, {'name': 'Mag...","[{'name': 'Phy', 'amount': 49}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'D'}, {'name': 'De...","[{'name': 'Str', 'amount': 14}, {'name': 'Dex'...",Axe,5.5,...,NaN,NaN,NaN,NaN,49,33,33,33,33,36
2,17f695c42f0l0i1ohb4cao0qxackpu,Forked Hatchet,https://eldenring.fanapis.com/images/weapons/1...,Peculiar hatchet wielded by imps. The gently u...,"[{'name': 'Phy', 'amount': 101}, {'name': 'Mag...","[{'name': 'Phy', 'amount': 40}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'D'}, {'name': 'De...","[{'name': 'Str', 'amount': 9}, {'name': 'Dex',...",Axe,2.5,...,NaN,NaN,NaN,NaN,40,27,27,27,27,27
3,17f69c6388cl0i1ohcb8denrbpuhqs,Iron Cleaver,https://eldenring.fanapis.com/images/weapons/1...,"Fairly large iron cleaver, commonly used by th...","[{'name': 'Phy', 'amount': 120}, {'name': 'Mag...","[{'name': 'Phy', 'amount': 51}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'C'}, {'name': 'De...","[{'name': 'Str', 'amount': 15}, {'name': 'Dex'...",Axe,6.0,...,NaN,NaN,NaN,NaN,51,34,34,34,34,34
4,17f699dbb9dl0i1ohfs2g1wqxwja74,Battle Axe,https://eldenring.fanapis.com/images/weapons/1...,"A thick, heavy blade attached to a handle. A v...","[{'name': 'Phy', 'amount': 123}, {'name': 'Mag...","[{'name': 'Phy', 'amount': 47}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'D'}, {'name': 'De...","[{'name': 'Str', 'amount': 12}, {'name': 'Dex'...",Axe,4.5,...,NaN,NaN,NaN,NaN,47,31,31,31,31,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,17f69571714l0i1ptc8s1fsagr8zf,Thorned Whip,https://eldenring.fanapis.com/images/weapons/1...,Hefty whip coverd in crimson thorns. Weapon of...,"[{'name': 'Phy', 'amount': 102}, {'name': 'Mag...","[{'name': 'Phy', 'amount': 26}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'E'}, {'name': 'De...","[{'name': 'Str', 'amount': 8}, {'name': 'Dex',...",Whip,2.5,...,NaN,NaN,NaN,NaN,26,15,15,15,15,15
303,17f69c5c0bfl0i1ptlhsrovoc8dx07,Magma Whip Candlestick,https://eldenring.fanapis.com/images/weapons/1...,This ritual implement is a three pronged candl...,"[{'name': 'Phy', 'amount': 72}, {'name': 'Mag'...","[{'name': 'Phy', 'amount': 21}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'E'}, {'name': 'De...","[{'name': 'Str', 'amount': 8}, {'name': 'Dex',...",Whip,2.5,...,NaN,NaN,NaN,NaN,21,10,36,10,10,15
304,17f694eefe9l0i1pto8dn7ezy61a99,Hoslow's Petal Whip,https://eldenring.fanapis.com/images/weapons/1...,Metal whip formed of razor-sharp chain-link bl...,"[{'name': 'Phy', 'amount': 110}, {'name': 'Mag...","[{'name': 'Phy', 'amount': 28}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'D'}, {'name': 'De...","[{'name': 'Str', 'amount': 10}, {'name': 'Dex'...",Whip,3.5,...,NaN,NaN,NaN,NaN,28,17,17,17,17,17
305,17f696af19bl0i1ptpv0fdawl72o4yq,Giant's Red Braid,https://eldenring.fanapis.com/images/weapons/1...,Hefty whip woven from the flame-red hair of a ...,"[{'name': 'Phy', 'amount': 82}, {'name': 'Mag'...","[{'name': 'Phy', 'amount': 24}, {'name': 'Mag'...","[{'name': 'Str', 'scaling': 'D'}, {'name': 'De...","[{'name': 'Str', 'amount': 18}, {'name': 'Dex'...",Whip,2.5,...,NaN,NaN,NaN,NaN,24,12,26,12,12,15


In [368]:
def parse_scaling(df):
    #takes in row passed in from weapons_df
    #loops through df["scalesWith"], matches and saves scaling in list
    #print(df)
    scales_with = []
    for i in df["scalesWith"]:
        #print(i)
        scales_with.append(re.findall(r"(?:[A-Z][a-z][a-z])", i))
    #print(scales_with)
    #loops through df["scalesWith"], matches and saves scalings in list
    clean_scalings = []
    for k in df["scalesWith"]:
        clean_scalings.append(re.findall(r"(?:[A-E])\'", k))

        #for i in scaling:
        #    clean_scalings.append(i[0])
    
    
    #print(clean_scalings)


    #lists = [scales_with,clean_scalings]  
    #df = pd.concat([pd.Series(x) for x in lists], axis=1, ignore_index=True)
    #df = df.T
    #print(df)
    
    #df.columns = df.iloc[0]
    #df = df[1:]
    #df["scalesWith"] = scales_with
    #df["scaling_rank"] = clean_scalings
    #df = df.rename(columns=df.iloc[0])[1:]
    #print(df)
    return (scales_with,clean_scalings)

def add_scaling(df):
    scale = parse_scaling(df)

    scale_with = scale[0]
    scaling_rate = scale[1]
    #df["scalesWith"] = "0"
    df["scalesWith"] = scale_with
    df["scalingRate"] = scaling_rate
    return df
    